# Solution Optimization Evaluaton Raw TextGrad

In [42]:
import pandas as pd
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifier
from textgrad.loss import TextLoss

## Load Datasets

In [43]:
initial_solution = pd.read_csv("csv/initial_solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,D,GPQA-Diamond,-
1,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-


In [44]:
# Test size only 50 rows each datasets (Total 150 rows)

df_gpqa = initial_solution[initial_solution['source'] == 'GPQA-Diamond'].head(50)
df_mmlu_ml = initial_solution[initial_solution['source'] == 'MMLU-ML'].head(50)
df_mmlu_cp = initial_solution[initial_solution['source'] == 'MMLU-CP'].head(50)
df_test = pd.concat([df_gpqa, df_mmlu_ml, df_mmlu_cp], ignore_index=True)

df_test

,id,formatted_question,raw_solution,correct_answer,source,subject
0,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,D,GPQA-Diamond,-
1,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-


## Experiment

In [45]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

In [46]:
def evaluate_with_raw_textgrad(row_data):
    match = initial_solution[initial_solution["id"] == row_data["id"]]
    if match.empty:
        return None  # or raise error
    formatted_question = match.iloc[0]["formatted_question"]
    result = {
        "id": row_data["id"],
        "raw_solution": row_data["raw_solution"],
        "correct_answer": row_data["correct_answer"],
        "source": row_data["source"],
        "subject": row_data["subject"]
    }
    
    solution = Variable(row_data["raw_solution"],
                    requires_grad=True,
                    role_description=f"Solution to the math question: {formatted_question}")
    loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                    Do not attempt to solve it yourself, do not give a solution, 
                                    only identify errors. Be super concise.""",
                                    requires_grad=False,
                                    role_description="system prompt")
    optimizer = TextualGradientDescent([solution])
    loss = TextLoss(loss_system_prompt, engine=engine)
    
    # Iterate 5 times
    for i in range(1, 6):
        optimizer.zero_grad()  # Clean gradients
        loss_result = loss(solution)
        
        loss_result.backward()
        optimizer.step()
        result[f"solution_{i}"] = solution.value

    return result

## Running Evaluation

### Raw TextGrad

In [47]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

results = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=32) as executor:
    # Submit all tasks
    futures = [
        executor.submit(evaluate_with_raw_textgrad, row.to_dict()) 
        for _, row in df_test.iterrows()
    ]
    
    # Use tqdm for progress tracking
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        result = future.result()
        if result is not None:
            results.append(result)

raw_textgrad = pd.DataFrame(results)

print(f"Completed in {time.time() - start_time:.1f} seconds")
raw_textgrad.to_csv('results/raw_textgrad.csv', index=False)

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> Here's how we can determine the number of carbon atoms in product 3:\n\n1. **Reaction 1:** trans-Cinnamaldehyde (C9H8O) reacts with methylmagnesium bromide (CH3MgBr), a Grignard reagent.  Grignard reagents add to the carbonyl carbon, forming an alcohol.  This adds one carbon atom (from the methyl group) to the molecule.  So, product 1 has 9 + 1 = 10 carbon atoms.\n\n2. **Reaction 2:** Product 1 (the secondary alcohol) is treated with pyridinium chlorochromate (PCC). PCC is an oxidizing agent that converts secondary alcohols to ketones.  The number of carbon atoms remains the same. So, product 2 still has 10 carbon atoms.\n\n3. **Reaction 3:** Product 2 (the ketone) is t

Processing:  50%|█████     | 1/2 [00:47<00:47, 47.80s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The energy-time uncertainty principle states that ΔE * Δt ≥ ħ/2. To clearly distinguish between two energy levels, the energy difference must be greater than the uncertainty in energy (ΔE).  Since we want to *clearly* resolve the levels, we should use the *larger* lifetime to determine the minimum resolvable energy difference. The larger the lifetime, the smaller the uncertainty in energy, and thus the smaller the energy difference required to distinguish the levels.\n\nGiven lifetimes of 10⁻⁹ s and 10⁻⁸ s, the larger lifetime is 10⁻⁸ s. Using this lifetime:\n\nΔE ≥ ħ/(2*Δt)\nħ ≈ 6.582 * 10⁻¹⁶ eV*s\nΔt = 10⁻⁸ s\n\nΔE ≥ (6.582 * 10⁻¹⁶ eV*s) / (2 * 10⁻⁸ s)\nΔE ≥ 3.291 * 1

Processing: 100%|██████████| 2/2 [01:01<00:00, 30.81s/it]

Completed in 61.6 seconds
